In [1]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import  time 
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.nn.modules.loss import _Loss 
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *

In [2]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [3]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

C:\Users\prath\anaconda3\Lib\site-packages\torch\__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
dtype = 'float32'

In [15]:
training_x=[]
path='./test/input/'
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
for item in path_list:
    impath=path+item
    #print("开始处理"+impath)
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    training_x.append(imgx)   

X_train = []

for features in training_x:
    X_train.append(features)
    
X_train = np.array(X_train)

In [16]:
X_train=X_train.astype(dtype)
X_train= torch.from_numpy(X_train)
X_train=X_train.permute(0,3,1,2)

In [17]:
X_train=X_train/255.0
X_train.shape

torch.Size([108, 3, 256, 256])

In [18]:
training_y=[]
path='./test/GT/'
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
for item in path_list:
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    training_y.append(imgx)
    
    
    

y_train = []

for features in training_y:
    y_train.append(features)
    

    
y_train = np.array(y_train)


In [19]:
y_train=y_train.astype(dtype)
y_train= torch.from_numpy(y_train)
y_train=y_train.permute(0,3,1,2)

In [20]:
y_train=y_train/255.0
y_train.shape

torch.Size([108, 3, 256, 256])

In [21]:
test_x=[]
path='./test/input/'
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
for item in path_list:
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    test_x.append(imgx)
    
    
    

x_test = []

for features in test_x:
    x_test.append(features)
    

    
x_test = np.array(x_test)

In [22]:
x_test=x_test.astype(dtype)
x_test= torch.from_numpy(x_test)
x_test=x_test.permute(0,3,1,2)

In [23]:
x_test=x_test/255.0

In [24]:
x_test.shape

torch.Size([108, 3, 256, 256])

In [25]:
test_Y=[]
path='./test/GT/'
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
for item in path_list:
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx=cv2.resize(imgx,(256,256))
    test_Y.append(imgx)
    
    
    

Y_test = []

for features in test_Y:
    Y_test.append(features)
    

    
Y_test = np.array(Y_test)
#X_train = np.array(X_train)

In [26]:
Y_test=Y_test.astype(dtype)
Y_test= torch.from_numpy(Y_test)
Y_test=Y_test.permute(0,3,1,2)

In [27]:
Y_test=Y_test/255.0

In [28]:
Y_test.shape

torch.Size([108, 3, 256, 256])

In [29]:
import torch.utils.data as dataf
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


dataset = dataf.TensorDataset(X_train,y_train)
loader = dataf.DataLoader(dataset, batch_size=1, shuffle=True,num_workers=4)

In [32]:
criterion_GAN = nn.MSELoss(size_average=False)
criterion_pixelwise = nn.MSELoss(size_average=False)
MSE = nn.MSELoss(size_average=False)
SSIM = pytorch_ssim.SSIM()
L_vgg = VGG19_PercepLoss()
L_lab = lab_Loss()
L_lch = lch_Loss()

# Remove .cuda() calls
# criterion_pixelwise.cuda()
# L_vgg.cuda()
# MSE.cuda()
# SSIM.cuda()
# criterion_GAN.cuda()
# L_lab.cuda()
# L_lch.cuda()


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\prath/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|███████████████████████████████████████████████████████████████████████████████| 548M/548M [02:04<00:00, 4.60MB/s]


In [33]:
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel=0.1
lambda_lab=0.001
lambda_lch=1
lambda_con = 100
lambda_ssim=100

In [34]:
# Calculate output of image discriminator (PatchGAN)
patch = (1, 256 // 2 ** 5, 256// 2 ** 5)

In [35]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [36]:

use_pretrain=False
if use_pretrain:
    # Load pretrained models
    start_epoch=0
    generator.load_state_dict(torch.load("saved_models/G/generator_%d.pth" % (start_epoch)))
    discriminator.load_state_dict(torch.load("saved_models/D/discriminator_%d.pth" % (start_epoch)))
    print('successfully loading epoch {} 成功！'.format(start_epoch))
else:
    start_epoch = 0
    print('No pretrain model found, training will start from scratch！')

No pretrain model found, training will start from scratch！


In [37]:
def sample_images(batches_done):
    """Saves a generated sample from the validation set"""
    generator.eval()
    i=random.randrange(1,90)
    real_A = Variable(x_test[i,:,:,:]).cuda()
    real_B = Variable(Y_test[i,:,:,:]).cuda()
    real_A=real_A.unsqueeze(0)
    real_B=real_B.unsqueeze(0)
    fake_B = generator(real_A)
    #print(fake_B.shape)
    imgx=fake_B[3].data
    imgy=real_B.data
    x=imgx[:,:,:,:]
    y=imgy[:,:,:,:]
    img_sample = torch.cat((x,y), -2)
    save_image(img_sample, "images/%s/%s.png" % ('results', batches_done), nrow=5, normalize=True)#要改

In [38]:
from torch.optim import lr_scheduler
LR=0.0005

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=LR,  betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=LR,  betas=(0.5, 0.999))
scheduler_G=lr_scheduler.StepLR(optimizer_G,step_size=40,gamma=0.8)
scheduler_D=lr_scheduler.StepLR(optimizer_D,step_size=40,gamma=0.8)

In [39]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor